# Tailpatch scores

In [ ]:
## TailPatch Function: With Log Likelihood
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torch.nn as nn
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.optim import AdamW
import numpy as np
import random
import gc
import math
from torch.utils.data import TensorDataset



import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
device = torch.device("cpu")

import random
import numpy as np
import torch

def tp_debug_log(samples,output_ids, prompt_length=1):
    SEED = 42
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)  

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    dataloader = DataLoader(
        samples,
        batch_size=1,
        shuffle=False
    )

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    tokenizer.pad_token = tokenizer.eos_token
    torch.manual_seed(42)
    torch.cuda.manual_seed(42)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    device = 'cpu'
    model = GPT2LMHeadModel.from_pretrained('out/gpt2-scratch-mixed')
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=1e-5)
    with torch.no_grad():
        model_pass = model(output_ids, labels=output_ids)

    logits = model_pass.logits
    logits = logits[:, prompt_length-1:-1, :]
    log_probs = torch.log_softmax(logits, dim=-1)
    target_ids = output_ids[:, prompt_length:]
    token_probs = log_probs.gather(2, target_ids.unsqueeze(-1))
    token_probs = token_probs.squeeze(-1)
    print(tokenizer.decode(target_ids[0], skip_special_tokens=True))
    original_probability = 0
    for i in range(token_probs.shape[1]):
        original_probability += token_probs[0,i]
    original_probability = math.exp(original_probability.item())
    print('Original Probability:', original_probability)
    original_probs = token_probs.clone()



    model.train()
    optimizer = AdamW(model.parameters(), lr=1e-5)
    optimizer.zero_grad()
    for i, batch in enumerate(dataloader):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = input_ids.clone()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()

        del input_ids, attention_mask, outputs, labels, loss
        gc.collect()
        torch.cuda.empty_cache()

    optimizer.step()
    optimizer.zero_grad()

    model.eval()
    
    with torch.no_grad():
        model_pass = model(output_ids, labels=output_ids)

    logits = model_pass.logits
    logits = logits[:, prompt_length-1:-1, :]
    log_probs = torch.log_softmax(logits, dim=-1)
    target_ids = output_ids[:, prompt_length:]
    token_probs = log_probs.gather(2, target_ids.unsqueeze(-1))
    token_probs = token_probs.squeeze(-1)
    probability = 1
    for i in range(token_probs.shape[1]):
        probability += token_probs[0,i]
    probability = math.exp(probability.item())   
    return probability, original_probability, token_probs, original_probs

In [ ]:
from datasets import load_from_disk
train_dataset = load_from_disk("data/training_data/tokenized_wit_dataset/train")
eval_dataset = load_from_disk("data/training_data/tokenized_wit_dataset/test")
from rank_bm25 import BM25Okapi
import torch
corpus = torch.load('data/training_data/untokenized_wiki.pt', map_location='cpu')
print(f"Loaded {len(corpus)} documents.")
print("Sample:", corpus[0][:300])
corpus = np.array(corpus)
print(len(corpus))
tokenized_corpus = [doc.lower().split() for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)


In [ ]:
examples = [
    'ancient_rome',
    'impressionism',
    'ww2',
    'dna',
    'solar_system',
    'philosophy_mind',
    'jazz',
    'thermodynamics',
    'internet',
    'feminism',
    'everest',
    'iss',
    'ww1',
    'ancient_egypt',
    'ancient_greece',
    'art_deco',
    'big_bang',
    'buddhism',
    'democracy',
    'ecology',
    'genetics',
    'gothic_architecture',
    'probability',
    'renaissance',
    'shakespeare'
]
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import pandas as pd
import numpy as np
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
dfs = []
for example in examples[13:]:
    print(example)
    ckpt = torch.load(f'out/wiki_models_finetuned/fisher_regularized_models/{example}_finetuned_fisher.pt', map_location='cpu')
    output_ids = ckpt['output_ids']
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    print(output_text)
    splits = output_text.split('.')
    prompt = splits[0]+'.'
    print(prompt)
    prompt_length = tokenizer.encode(prompt, add_special_tokens=False)
    prompt_length = len(prompt_length)
    print(prompt_length)
    losses_af = np.load(f'data/losses/wiki/finetuned/fisher_regularized/{example}_finetuned.npy')
    losses_bf = np.load(f'data/losses/wiki/losses_bf.npy')
    losses_diff = losses_bf - losses_af
    sorted_indices = np.argsort(losses_diff)
    samples_ft = []
    for i in range(20):
        print(tokenizer.decode(train_dataset[int(sorted_indices[i])]['input_ids'], skip_special_tokens=True))
        print('----------------------------------------')
        sample_ft = {
            'input_ids': torch.tensor(train_dataset[int(sorted_indices[i])]['input_ids']),
            'attention_mask': torch.ones_like(torch.tensor(train_dataset[int(sorted_indices[i])]['input_ids'])),
            'labels': torch.tensor(train_dataset[int(sorted_indices[i])]['input_ids'])
        }
        samples_ft.append(sample_ft)
    p_1, original, _, _ = tp_debug_log(samples_ft[:1],output_ids=output_ids, prompt_length=prompt_length)
    p_3, original, _, _ = tp_debug_log(samples_ft[:3],output_ids=output_ids, prompt_length=prompt_length)
    p_5, original, _, _ = tp_debug_log(samples_ft[:5],output_ids=output_ids, prompt_length=prompt_length)
    p_7, original, _, _ = tp_debug_log(samples_ft[:7],output_ids=output_ids, prompt_length=prompt_length)
    p_10, original, _, _ = tp_debug_log(samples_ft[:10],output_ids=output_ids, prompt_length=prompt_length)
    p_15, original, _, _ = tp_debug_log(samples_ft[:15],output_ids=output_ids, prompt_length=prompt_length)
    p_20, original, _, _ = tp_debug_log(samples_ft[:20],output_ids=output_ids, prompt_length=prompt_length)
    dfs.append({
        'method': 'Finetuning',
        'name': example,
        '1': p_1,
        '3': p_3,
        '5': p_5,
        '7': p_7,
        '10': p_10,
        '15': p_15,
        '20': p_20
    })
    dfs.append({
        'method': 'Original',
        'name': example,
        '1': original,
        '3': original,
        '5': original,
        '7': original,
        '10': original,
        '15': original,
        '20': original
    })
    
    print('----------------------------------------')
    losses_unlearned = np.load(f'data/losses/wiki/unlearned/fisher_regularized/{example}_unlearned.npy')
    losses_diff_unlearned = np.abs(losses_af - losses_unlearned)
    sorted_indices_unlearned = np.argsort(losses_diff_unlearned)[::-1]
    samples_un_ft = []
    for i in range(20):
        print(tokenizer.decode(train_dataset[int(sorted_indices_unlearned[i])]['input_ids'], skip_special_tokens=True))
        print('----------------------------------------')
        sample_un_ft = {
            'input_ids': torch.tensor(train_dataset[int(sorted_indices_unlearned[i])]['input_ids']),
            'attention_mask': torch.ones_like(torch.tensor(train_dataset[int(sorted_indices_unlearned[i])]['input_ids'])),
            'labels': torch.tensor(train_dataset[int(sorted_indices_unlearned[i])]['input_ids'])
        }
        samples_un_ft.append(sample_un_ft)
    p_1_un, original_un, _, _ = tp_debug_log(samples_un_ft[:1],output_ids=output_ids, prompt_length=prompt_length)
    p_3_un, original_un, _, _ = tp_debug_log(samples_un_ft[:3],output_ids=output_ids, prompt_length=prompt_length)
    p_5_un, original_un, _, _ = tp_debug_log(samples_un_ft[:5],output_ids=output_ids, prompt_length=prompt_length)
    p_7_un, original_un, _, _ = tp_debug_log(samples_un_ft[:7],output_ids=output_ids, prompt_length=prompt_length)
    p_10_un, original_un, _, _ = tp_debug_log(samples_un_ft[:10],output_ids=output_ids, prompt_length=prompt_length)
    p_15_un, original_un, _, _ = tp_debug_log(samples_un_ft[:15],output_ids=output_ids, prompt_length=prompt_length)
    p_20_un, original_un, _, _ = tp_debug_log(samples_un_ft[:20],output_ids=output_ids, prompt_length=prompt_length)
    dfs.append({
        'method': 'FT & UN',
        'name': example,
        '1': p_1_un,
        '3': p_3_un,
        '5': p_5_un,
        '7': p_7_un,
        '10': p_10_un,
        '15': p_15_un,
        '20': p_20_un
    })


    losses_diff = losses_bf - losses_unlearned
    sorted_indices_un = np.argsort(losses_diff)
    samples_un = []
    for i in range(20):
        print(tokenizer.decode(train_dataset[int(sorted_indices_un[i])]['input_ids'], skip_special_tokens=True))
        print('----------------------------------------')
        sample_un = {
            'input_ids': torch.tensor(train_dataset[int(sorted_indices_un[i])]['input_ids']),
            'attention_mask': torch.ones_like(torch.tensor(train_dataset[int(sorted_indices_un[i])]['input_ids'])),
            'labels': torch.tensor(train_dataset[int(sorted_indices_un[i])]['input_ids'])
        }
        samples_un.append(sample_un)
    p_1_un, original_un, _, _ = tp_debug_log(samples_un[:1],output_ids=output_ids, prompt_length=prompt_length)
    p_3_un, original_un, _, _ = tp_debug_log(samples_un[:3],output_ids=output_ids, prompt_length=prompt_length)
    p_5_un, original_un, _, _ = tp_debug_log(samples_un[:5],output_ids=output_ids, prompt_length=prompt_length)
    p_7_un, original_un, _, _ = tp_debug_log(samples_un[:7],output_ids=output_ids, prompt_length=prompt_length)
    p_10_un, original_un, _, _ = tp_debug_log(samples_un[:10],output_ids=output_ids, prompt_length=prompt_length)
    p_15_un, original_un, _, _ = tp_debug_log(samples_un[:15],output_ids=output_ids, prompt_length=prompt_length)
    p_20_un, original_un, _, _ = tp_debug_log(samples_un[:20],output_ids=output_ids, prompt_length=prompt_length)
    dfs.append({
        'method': 'Unlearning',
        'name': example,
        '1': p_1_un,
        '3': p_3_un,
        '5': p_5_un,
        '7': p_7_un,
        '10': p_10_un,
        '15': p_15_un,
        '20': p_20_un
    })
    print('----------------------------------------')
    query = tokenizer.decode(output_ids[0, prompt_length:], skip_special_tokens=True)
    print(query)
    tokenized_query = query.lower().split()
    scores = bm25.get_scores(tokenized_query)
    top_n = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)[:20]
    samples_bm25 = []
    for rank, (idx, score) in enumerate(top_n, 1):
        print(f"\nTop {rank} (index: {idx}, score: {score:.2f}):\n{corpus[idx][:300]}")
        print(tokenizer.decode(train_dataset[idx]['input_ids'], skip_special_tokens=True))
        sample_bm25 = {
            'input_ids': torch.tensor(train_dataset[idx]['input_ids']),
            'attention_mask': torch.ones_like(torch.tensor(train_dataset[idx]['input_ids'])),
            'labels': torch.tensor(train_dataset[idx]['input_ids'])
        }
        samples_bm25.append(sample_bm25)

    p_1, original, _, _ = tp_debug_log(samples_bm25[:1],output_ids=output_ids, prompt_length=prompt_length)
    p_3, original, _, _ = tp_debug_log(samples_bm25[:3],output_ids=output_ids, prompt_length=prompt_length)
    p_5, original, _, _ = tp_debug_log(samples_bm25[:5],output_ids=output_ids, prompt_length=prompt_length)
    p_7, original, _, _ = tp_debug_log(samples_bm25[:7],output_ids=output_ids, prompt_length=prompt_length)
    p_10, original, _, _ = tp_debug_log(samples_bm25[:10],output_ids=output_ids, prompt_length=prompt_length)
    p_15, original, _, _ = tp_debug_log(samples_bm25[:15],output_ids=output_ids, prompt_length=prompt_length)
    p_20, original, _, _ = tp_debug_log(samples_bm25[:20],output_ids=output_ids, prompt_length=prompt_length)

    dfs.append({
        'name': example,
        'method': 'BM25',
        '1': p_1,
        '3': p_3,
        '5': p_5,
        '7': p_7,
        '10': p_10,
        '15': p_15,
        '20': p_20
    })

    trackstar_influence = np.load(f'data/training_data/trackstar/wiki/testing/gradients/influence_list_{example}_dot_model.npy')
    print(trackstar_influence.shape)
    trackstar_influence = trackstar_influence[:len(train_dataset)] ## When running in chunks, last couple items are empty
    sorted_indices = np.argsort(trackstar_influence)[::-1]
    samples_ft = []
    for i in range(20):
        sample_ft = {
            'input_ids': torch.tensor(train_dataset[int(sorted_indices[i])]['input_ids']),
            'attention_mask': torch.ones_like(torch.tensor(train_dataset[int(sorted_indices[i])]['input_ids'])),
            'labels': torch.tensor(train_dataset[int(sorted_indices[i])]['input_ids'])
        }
        samples_ft.append(sample_ft)
    
    p_1, original, _, _ = tp_debug_log(samples_ft[:1],output_ids=output_ids, prompt_length=prompt_length)
    p_3, original, _, _ = tp_debug_log(samples_ft[:3],output_ids=output_ids, prompt_length=prompt_length)
    p_5, original, _, _ = tp_debug_log(samples_ft[:5],output_ids=output_ids, prompt_length=prompt_length)
    p_7, original, _, _ = tp_debug_log(samples_ft[:7],output_ids=output_ids, prompt_length=prompt_length)
    p_10, original, _, _ = tp_debug_log(samples_ft[:10],output_ids=output_ids, prompt_length=prompt_length)
    p_15, original, _, _ = tp_debug_log(samples_ft[:15],output_ids=output_ids, prompt_length=prompt_length)
    p_20, original, _, _ = tp_debug_log(samples_ft[:20],output_ids=output_ids, prompt_length=prompt_length)
    dfs.append({
        'method': 'TrackStar',
        'name': example,
        '1': p_1,
        '3': p_3,
        '5': p_5,
        '7': p_7,
        '10': p_10,
        '15': p_15,
        '20': p_20
    })
    random_samples = np.random.choice(len(train_dataset), size=20)
    samples_random = []
    for sample in random_samples:
        print(tokenizer.decode(train_dataset[int(sample)]['input_ids'], skip_special_tokens=True))
        samples_random.append({
            'input_ids': torch.tensor(train_dataset[int(sample)]['input_ids']),
            'attention_mask': torch.ones_like(torch.tensor(train_dataset[int(sample)]['input_ids'])),
            'labels': torch.tensor(train_dataset[int(sample)]['input_ids'])
        })
    p_1, original, _, _ = tp_debug_log(samples_random[:1],output_ids=output_ids, prompt_length=prompt_length)
    p_3, original, _, _ = tp_debug_log(samples_random[:3],output_ids=output_ids, prompt_length=prompt_length)
    p_5, original, _, _ = tp_debug_log(samples_random[:5],output_ids=output_ids, prompt_length=prompt_length)
    p_7, original, _, _ = tp_debug_log(samples_random[:7],output_ids=output_ids, prompt_length=prompt_length)
    p_10, original, _, _ = tp_debug_log(samples_random[:10],output_ids=output_ids, prompt_length=prompt_length)
    p_15, original, _, _ = tp_debug_log(samples_random[:15],output_ids=output_ids, prompt_length=prompt_length)
    p_20, original, _, _ = tp_debug_log(samples_random[:20],output_ids=output_ids, prompt_length=prompt_length)
    dfs.append({
        'method': 'Random',
        'name': example,
        '1': p_1,
        '3': p_3,
        '5': p_5,
        '7': p_7,
        '10': p_10,
        '15': p_15,
        '20': p_20
    })
    total_df = pd.DataFrame(dfs)
    os.makedirs('data/results', exist_ok=True)
    total_df.to_csv(f'data/results/results_aggregated.csv', index=False)
    print('\n\n\n\n')
total_df = pd.DataFrame(dfs)
os.makedirs('data/results', exist_ok=True)
total_df.to_csv('data/results/results_aggregated.csv', index=False)



In [ ]:
import pandas as pd
df = pd.read_csv('../data/results/results_aggregated.csv')
df

In [ ]:
import pandas as pd
import numpy as np


cols_to_check = ['1', '3', '5', '7', '10', '15', '20']

# Ensure those columns are strings, if they are originally float or int
df.columns = df.columns.astype(str)

results = []

for namegrouped, group in df.groupby('name'):
    original_row = group[group['method'].str.lower() == 'original']
    if original_row.empty:
        print(namegrouped)
        continue
    original_values = original_row[cols_to_check].iloc[0]
    
    
    for _, row in group.iterrows():
        if row['method'].lower() == 'original':
            continue
        rel_increase =np.abs(((row[cols_to_check].astype(float)) - (original_values.astype(float))) / np.abs(original_values.astype(float))) * 100
        result_row = {
            'name': namegrouped,
            'method': row['method'],
        }
        for col in cols_to_check:
            result_row[f'rel_increase_{col}'] = rel_increase[col]
        results.append(result_row)

# Final result
rel_df = pd.DataFrame(results)

rel_df

In [ ]:
avg_df = (
    rel_df.groupby(['method'])
    .agg({
        'rel_increase_1': ['mean', 'std'],
        'rel_increase_3': ['mean', 'std'],
        'rel_increase_5': ['mean', 'std'],
        'rel_increase_7': ['mean', 'std'],
        'rel_increase_10': ['mean', 'std'],
        'rel_increase_15': ['mean', 'std'],
        'rel_increase_20': ['mean', 'std'],
    })
)
avg_df = avg_df.round(2)
avg_df



In [ ]:
import pandas as pd
df_cleaned = avg_df.loc[:, avg_df.columns.get_level_values(1) == 'mean']
df_cleaned.columns = [col[0].split('_')[-1] for col in df_cleaned.columns]

df_cleaned.columns.name = None
df_cleaned.index.name = None
df_cleaned = df_cleaned.sort_values(by=['1'], ascending=True)
df_cleaned.rename(index={'FT/UN': 'FT & UN'}, inplace=True)
df_cleaned_original = pd.concat([df_cleaned.loc[["BM25"]], df_cleaned.drop(index="BM25")])
#df_cleaned.to_csv('avg_df_fisher_raw_.csv', index=True)
df_cleaned_original



In [ ]:
# Read the data
df_1 = pd.read_csv('../data/results/avg_results_with_gecko.csv', index_col=0)
df_2 = pd.read_csv('../data/results/avg_df_gutenberg_raw.csv', index_col=0)
df_1 = df_1.drop(columns=['num_steps'])
df_2.rename(index={'UN - BF': 'Unlearning'}, inplace=True)
df_2.rename(index={'BF - FT': 'Finetuning'}, inplace=True)
combined = pd.concat([df_1, df_2], axis=1, keys=["Wikipedia", "Gutenberg"])

combined

In [ ]:
import pandas as pd

# Read
df_1 = pd.read_csv('../data/results/avg_results_with_gecko.csv', index_col=0)
df_2 = pd.read_csv('../data/results/avg_df_gutenberg_gecko.csv', index_col=0)

# Keep only the models of interest (by index) in df_1
wanted = ['BM25', 'TrackStar', 'Random', 'FT & UN', 'Gecko']
df_1 = df_1.loc[df_1.index.intersection(wanted)]
# (optional) order rows as in `wanted`
df_1 = df_1.reindex(wanted).dropna(how='all')

# Drop extra column present only in df_1
if 'num_steps' in df_1.columns:
    df_1 = df_1.drop(columns=['num_steps'])

# Harmonize names in df_2
df_2 = df_2.rename(index={'UN - BF': 'Unlearning', 'BF - FT': 'Finetuning'})

# (optional) align df_2 to a desired order; adapt as needed
order_gut = ['BM25', 'TrackStar', 'Random', 'FT & UN', 'Gecko']
df_2 = df_2.loc[df_2.index.intersection(order_gut)].reindex(order_gut).dropna(how='all')

# Combine with MultiIndex columns
combined = pd.concat([df_1, df_2], axis=1, keys=["Wikipedia", "Gutenberg"])

combined


In [ ]:
import pandas as pd





# Define levels: Dataset, Top-k (%), and specific k values
k_vals = [1, 3, 5, 7, 10, 15, 20]
datasets = ["Wikipedia", "Gutenberg"]
topk_level = ["Top-k (\\%)"] * len(k_vals) * len(datasets)
dataset_level = sum([[d] * len(k_vals) for d in datasets], [])
k_level = k_vals * len(datasets)

# Assign new MultiIndex
combined.columns = pd.MultiIndex.from_arrays([dataset_level, topk_level, k_level])

def format_latex(df):
    df_fmt = df.copy().astype(str)
    datasets = df.columns.levels[0]
    topks = df.columns.levels[2]

    for dataset in datasets:
        for k in topks:
            col_vals = df[(dataset, "Top-k (\\%)", k)]
            sorted_vals = col_vals.sort_values(ascending=False)
            if len(sorted_vals) < 2:
                continue
            top1, top2 = sorted_vals.index[0], sorted_vals.index[1]
            for idx in col_vals.index:
                val = col_vals.loc[idx]
                if idx == top1:
                    df_fmt.loc[idx, (dataset, "Top-k (\\%)", k)] = f"\\textbf{{{val:.1f}}}"
                elif idx == top2:
                    df_fmt.loc[idx, (dataset, "Top-k (\\%)", k)] = f"\\underline{{{val:.1f}}}"
                else:
                    df_fmt.loc[idx, (dataset, "Top-k (\\%)", k)] = f"{val:.1f}"
    return df_fmt

formatted_df = format_latex(combined)
latex_code = formatted_df.to_latex(
    escape=False,
    multicolumn=True,
    multirow=True,
    column_format='l' + 'r'*7 + '|' + 'r'*7,
    index_names=False
)


# Replace left-aligned dataset headers with centered ones
latex_code = latex_code.replace(
    r'\multicolumn{7}{l}{Wikipedia}', r'\multicolumn{7}{c}{\textbf{Wikipedia}}'
)
latex_code = latex_code.replace(
    r'\multicolumn{7}{l}{Gutenberg}', r'\multicolumn{7}{c}{\textbf{Gutenberg}}'
)

# Print LaTeX
print(latex_code)
